In [1]:
import json
import torch
import random
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

/home/mgh23/Miniconda3/miniconda3/envs/ragattack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load 10k embeddings and their doc IDs
corpus_embeddings = torch.load("corpus_embeddings_10000.pt")
with open("corpus_ids_10000.json", "r") as f:
    corpus_ids = json.load(f)
corpus_id_set = set(corpus_ids)

/tmp/ipykernel_3465518/1079491128.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  corpus_embeddings = torch.load("corpus_embeddings_10000.pt")


In [3]:
contriever_tokenizer = AutoTokenizer.from_pretrained("facebook/contriever")
contriever_model = AutoModel.from_pretrained("facebook/contriever")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
contriever_model = contriever_model.to(device).eval()

In [ ]:
# bitsandbytes config if used quantization
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer and model from local cache
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

llama_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quant_config
).eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.81s/it]


In [ ]:
def select_relevant_queries(n=1, queries_path="./nq/queries.jsonl", qrels_path="./nq/qrels/test.tsv", corpus_path="./nq/corpus.jsonl", corpus_id_set=None):
    """
    Selects `n` queries with at least one relevant document in the top 10k embedded corpus.

    Args:
        n (int): Number of relevant queries to select.
        queries_path (str): Path to the queries JSONL file.
        qrels_path (str): Path to the qrels TSV file.
        corpus_path (str): Path to the corpus JSONL file.
        corpus_id_set (set): Set of document IDs included in the top 10k embedded corpus.

    Returns:
        List[Dict]: List of dictionaries with selected queries and their relevant docs.
    """

    # Load queries
    with open(queries_path, "r") as f:
        queries = [json.loads(line) for line in f]

    # Load qrels
    qrels = pd.read_csv(qrels_path, sep="\t")

    # Load and index corpus docs that are in the top 10k
    corpus_index = {}
    with open(corpus_path, "r") as f:
        for line in f:
            doc = json.loads(line)
            if doc["_id"] in corpus_id_set:
                corpus_index[doc["_id"]] = doc["text"]

    # Select queries with at least one relevant doc in the top 10k
    selected = []
    tried = set()

    while len(selected) < n and len(tried) < len(queries):
        query = random.choice(queries)
        query_id = query["_id"]

        if query_id in tried:
            continue
        tried.add(query_id)

        relevant_docs = qrels[qrels["query-id"] == query_id]["corpus-id"].tolist()
        relevant_in_10k = [doc_id for doc_id in relevant_docs if doc_id in corpus_index]

        if relevant_in_10k:
            selected.append({
                "query_id": query_id,
                "query_text": query["text"],
                "relevant_docs": [(doc_id, corpus_index[doc_id]) for doc_id in relevant_in_10k]
            })

    return selected


In [6]:
selected_queries = select_relevant_queries(n=10, corpus_id_set=corpus_id_set)

for entry in selected_queries:
    print(f"Query ID: {entry['query_id']}")
    print(f"Query: {entry['query_text']}")
    print("Relevant docs from top 10k:")
    for doc_id, doc_text in entry['relevant_docs']:
        print(f"\n[{doc_id}]\n{doc_text[:500]}...")
    print("\n" + "="*80 + "\n")


Query ID: test187
Query: who sings gimme some lovin in days of thunder
Relevant docs from top 10k:

[doc6891]
The song "Gimme Some Lovin'" is credited to Terry Reid, but the version in the movie is actually from The Spencer Davis Group. "Gimme Some Lovin'" also featured on Reid's 1991 solo album, The Driver, along with an alternate version of "The Last Note of Freedom" with different lyrics, titled "The Driver (Part 2)"....


Query ID: test7
Query: in order to prove disparate impact you first must establish
Relevant docs from top 10k:

[doc67]
A violation of Title VII of the 1964 Civil Rights Act may be proven by showing that an employment practice or policy has a disproportionately adverse effect on members of the protected class as compared with non-members of the protected class.[1] Therefore, the disparate impact theory under Title VII prohibits employers "from using a facially neutral employment practice that has an unjustified adverse impact on members of a protected class. A fac

In [ ]:
def generate_poison_text(Q, R, word_limit=100):
    """
    Generate a hallucinated passage I for a given question Q and target (false) answer R.
    The passage should be authoritative and misleading, consistent with PoisonedRAG setup.
    """

    prompt = f"""
    You are a helpful assistant.

    The user is asking the following question:
    "{Q}"

    Your task is to write a short, believable paragraph that would appear in a textbook or encyclopedia,
    but falsely claims that {R} is the correct answer to the question.

    The paragraph should sound authoritative and academic, and must not include any disclaimers or signs of fabrication.

    The paragraph should be {word_limit} words.

    Begin the hallucinated passage now:
    """

    inputs = llama_tokenizer(prompt, return_tensors="pt").to(llama_model.device)
    with torch.no_grad():
        output = llama_model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            repetition_penalty=1.1
        )

    generated = llama_tokenizer.decode(output[0], skip_special_tokens=True)
    I = generated.split("Begin the hallucinated passage now:")[-1].strip()
    return I

In [ ]:
poisoned_examples = []

for entry in selected_queries:
    Q = entry["query_text"]
    
    # Select a target (false) answer
    R = 'Elon Musk'
    
    I = generate_poison_text(Q, R)
    
    poisoned_examples.append({
        "query_id": entry["query_id"],
        "query_text": Q,
        "target_false_answer": R,
        "hallucinated_passage": I
    })

In [9]:
poisoned_embeddings = []
for ex in poisoned_examples:
    P = ex["query_text"] + " " + ex["hallucinated_passage"]

    inputs = contriever_tokenizer([P], padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        embedding = contriever_model(**inputs).last_hidden_state[:, 0].cpu()

    poisoned_embeddings.append({
        "query_id": ex["query_id"],
        "query_text": ex["query_text"],
        "target_false_answer": ex["target_false_answer"],
        "hallucinated_passage": ex["hallucinated_passage"],
        "embedding": embedding
    })

In [10]:
def embed_texts(texts):
    inputs = contriever_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = contriever_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0]  # CLS token
    return embeddings.cpu()

In [11]:
import torch.nn.functional as F

def evaluate_poisoned_ranking(poisoned_embeddings, corpus_embeddings, corpus_ids):
    """
    For each poisoned example, evaluate its similarity to the query using both
    dot product and cosine similarity, and rank it among the 10k corpus.

    Args:
        poisoned_embeddings (List[Dict]): List with query, poisoned text, and embeddings.
        corpus_embeddings (Tensor): Shape (N, d), unnormalized.
        corpus_ids (List[str]): List of document IDs in the same order as corpus_embeddings.

    Returns:
        List[Dict]: Results with dot and cosine ranks/scores for each poisoned doc.
    """
    corpus_norm = F.normalize(corpus_embeddings, dim=1)
    results = []

    for ex in poisoned_embeddings:
        query_embedding = embed_texts([ex["query_text"]])[0]  # (d,)
        poisoned_embedding = ex["embedding"].squeeze(0)       # (d,)

        # --- Normalize ---
        query_norm = F.normalize(query_embedding, dim=0)
        poison_norm = F.normalize(poisoned_embedding, dim=0)

        # --- Dot Product ---
        dot_scores = torch.matmul(corpus_embeddings, query_embedding)
        poison_dot_score = torch.dot(query_embedding, poisoned_embedding)
        poison_dot_rank = (dot_scores > poison_dot_score).sum().item() + 1

        # --- Cosine Similarity ---
        cos_scores = torch.matmul(corpus_norm, query_norm)
        poison_cos_score = torch.dot(poison_norm, query_norm)
        poison_cos_rank = (cos_scores > poison_cos_score).sum().item() + 1

        results.append({
            "query_id": ex["query_id"],
            "query_text": ex["query_text"],
            "poisoned_doc": ex["hallucinated_passage"],
            "poison_dot_score": poison_dot_score.item(),
            "poison_dot_rank": poison_dot_rank,
            "poison_cos_score": poison_cos_score.item(),
            "poison_cos_rank": poison_cos_rank
        })

    return results


In [12]:
ranking_results = evaluate_poisoned_ranking(poisoned_embeddings, corpus_embeddings, corpus_ids)

for res in ranking_results:
    print(f"Query ID: {res['query_id']}")
    print(f"Dot rank: {res['poison_dot_rank']} | score: {res['poison_dot_score']:.4f}")
    print(f"Cos rank: {res['poison_cos_rank']} | score: {res['poison_cos_score']:.4f}")
    print("-" * 60)


Query ID: test187
Dot rank: 4 | score: 2.3847
Cos rank: 6156 | score: 0.3477
------------------------------------------------------------
Query ID: test7
Dot rank: 44 | score: 2.7053
Cos rank: 7464 | score: 0.2521
------------------------------------------------------------
Query ID: test230
Dot rank: 1 | score: 2.5275
Cos rank: 6801 | score: 0.3175
------------------------------------------------------------
Query ID: test31
Dot rank: 2 | score: 3.5397
Cos rank: 1033 | score: 0.3564
------------------------------------------------------------
Query ID: test85
Dot rank: 7 | score: 2.7434
Cos rank: 218 | score: 0.3889
------------------------------------------------------------
Query ID: test252
Dot rank: 896 | score: 2.1785
Cos rank: 3516 | score: 0.4814
------------------------------------------------------------
Query ID: test13
Dot rank: 1 | score: 3.0546
Cos rank: 8272 | score: 0.2754
------------------------------------------------------------
Query ID: test92
Dot rank: 1 | score: